<a href="https://colab.research.google.com/github/phsoaressantos/infrahadoop/blob/main/Projeto_Infra_Hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://blog.infnet.com.br/wp-content/uploads/sites/50/2021/12/Substituir-Pexels-by-Anna-2048x1365.jpg" width="300" alt="cognitiveclass.ai logo" />
</center>

#**Pós Graduação em Engenharia de Dados: Big Data**

#**Projeto conclusão curso: Infra Hadoop**

Autor: Paulo Henrique Soares dos Santos



#**Dados**

## Visão geral dos dados

- Tabela Exportação:

CO_ANO = Year

CO_MES = Month

CO_NCM = NCM (Code description of the product - Mercosul Common Nomeclature)

CO_UNID = Statistics Unity Code

CO_PAIS = Code of the Destination Country

SG_UF_NCM = UF Code of the product origin

CO_VIA = Via of Transportation Code

CO_URF= URF Code of Embark and Disembark

QT_ESTAT = Statistic Unity

KG_LIQUIDO = Liquid Kg

VL_FOB = Value of Transaction (FOB Dolar US$)


- Tabela País:


CO_PAIS = Country Code

CO_PAIS_ISON3 = ISO 3 Country Code (Numerical)

CO_PAIS_ISOA3 = ISO 3 Country Code (Letters)

NO_PAIS = Country Name in Portugue (BR)

NO_PAIS_ING = Country Name in English

NO_PAIS_ESP = Country Name in Spanish

##Contexto

Este conjunto de dados é fornecido pelo Ministério da Indústria, Comércio Externo e Serviços do Brasil.

Neste site tem estatísticas de comércio exterior do Brasil. Que nos possibilita criar consultas detalhadas das exportações e importações brasileiras com as diversas variáveis da base de dados estatísticos.
Site: http://comexstat.mdic.gov.br/

Conteúdo

Estes dados vão de 1997 a Julho/2020 e são actualizados mensalmente no website comex stat.*1
Os dados são organizados pela NCM (Nomenclatura Comum para produtos de mercosul)


## Fonte Dados

Kaggle :

1 tabela (exportacao): https://www.kaggle.com/datasets/marcelopesse/brazil-exportimport-information?select=EXP_COMPLETA.csv *1

2 tabela (pais): https://www.kaggle.com/datasets/marcelopesse/brazil-exportimport-information?select=PAIS.csv

*1 Editei o CSV original devido ao tempo de upload na GCP, editei ate a linha 1.048.575.



#**Etapas do projeto** 

##Ambiente cloud
  Cluster criado na GCP utilizando DATAPROC - Cluster no computer engine com as seguintes configurações:

  *Nome: Cluster-b335-m

  *Local: us-central1

  *Tipo de Cluster: Padrão (1 mestre, N workers)

  *Tipo e versão imagem: 2.0debian10

  *Componentes adicionais: Todos, exceto Jupyter notebook

##Carga dados utilizando upload nativo

## Adcionando arquivos no HDFS
hdfs dfs -put PAIS.csv

hdfs dfs -put EXP_COMPLETA_2.csv

#**Hive**
##Acessando Hive
 #entrando no hive beeline, funcionou melhor e sem logs

beeline -u jdbc:hive2://localhost:10000/default -n paulo_hssantos@cluster-b335-m -d org.apache.hive.jdbc.HiveDriver

## Criando database trabalho:
  create database trabalho;

  use trabalho;

 #confere a criacao e utilização do db correto:

  show databases;

##Criando a minha TABELA COM EXPORTACAO 2

CREATE EXTERNAL TABLE exportacao(CO_ANO STRING, CO_MES INT, CO_NCM INT, CO_UNID INT, CO_PAIS STRING, SG_UF_NCM STRING, CO_VIA INT, CO_URF STRING, QT_ESTAT INT, KG_LIQUIDO INT, VL_FOB INT) 
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

  #conferindo tabela criada:

  show tables;

  describe exportacao;

##Criando a tabela pais
CREATE EXTERNAL TABLE pais(CO_PAIS STRING, CO_PAIS_ISON3 STRING, CO_PAIS_ISOA3 STRING, NO_PAIS STRING, NO_PAIS_ING STRING, NO_PAIS_ESP STRING) 
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
STORED AS TEXTFILE;

  #conferindo tabela criada:

  show tables;

  describe pais;

# Entrando com os dados nas tabelas pais e exportacao: também!

LOAD DATA INPATH '/user/paulo_hssantos/EXP_COMPLETA_2.csv' overwrite into table exportacao;

LOAD DATA INPATH '/user/paulo_hssantos/PAIS.csv' overwrite into table pais;



# Análise de dados

## Perguntas analiticas e respectivas consultas SQL:

1- Média de exportações por país de destino:

select co_pais, (SUM(VL_FOB)/COUNT(*)) AS CO_PAIS FROM trabalho.exportacao group by co_pais order by CO_PAIS;

2- Valor total de exportações em todo período analisado, agrupados por país de destino:

SELECT exportacao.CO_PAIS, NO_PAIS, SUM(VL_FOB)  from exportacao inner join pais on exportacao.CO_PAIS = pais.CO_PAIS group by NO_PAIS order by SUM(VL_FOB) desc;

3- Soma total de exportação por estado brasileiro;

SELECT SG_UF_NCM, SUM(VL_FOB) FROM trabalho.exportacao group by SG_UF_NCM;


4- Numero de exportações por estado brasileiro em ordem descendente;

select SG_UF_NCM,COUNT(*) AS numero_de_exportacoes from trabalho.exportacao GROUP BY SG_UF_NCM ORDER BY numero_de_exportacoes desc;

5- Os 100 primeiros Codigos NCM mais exportados nesta amostra de dados, ordenado por valor total. Informamos também a quantidade de exportações por NCM na coluna Numero Exportações.


SELECT CO_ANO, CO_NCM, SUM(VL_FOB) as TOTAL, count(*)as Numero_Exportacoes
FROM trabalho.exportacao
group by CO_ANO, CO_NCM 
order by SUM(VL_FOB) DESC
limit 100
;



#Linguagens e ferramentas utilizadas no projeto:

<p align="left">
<a href="https://cloud.google.com" target="_blank" rel="noreferrer"> <img src="https://www.vectorlogo.zone/logos/google_cloud/google_cloud-icon.svg" alt="gcp" width="40" height="40"/> </a>
<a href="https://hadoop.apache.org/" target="_blank" rel="noreferrer"> <img src="https://www.vectorlogo.zone/logos/apache_hadoop/apache_hadoop-icon.svg" alt="hadoop" width="40" height="40"/>
</a> <a href="https://hive.apache.org/" target="_blank" rel="noreferrer"> <img src="https://www.vectorlogo.zone/logos/apache_hive/apache_hive-icon.svg" alt="hive" width="40" height="40"/>
</a> <a href="https://www.mysql.com/" target="_blank" rel="noreferrer"> <img src="https://raw.githubusercontent.com/devicons/devicon/master/icons/mysql/mysql-original-wordmark.svg" alt="mysql" width="40" height="40"/> </a>  
</p>

# Contato: 

📧 paulosantos@id.uff.br